In [ ]:
from pathlib import Path
import json

In [ ]:
dataset_dir = Path("/mnt/ssd2/xin/repo/DART/Liebherr_Product")
repo_dir = Path("/mnt/ssd2/xin/repo/DART/diversification/dreambooth")

meta_dir = dataset_dir / "metadata"

image_dir = repo_dir / "generated_data"
meta_dir = dataset_dir / "metadata"
label_dir = dataset_dir / "labels_gen"
response_dir = repo_dir / "generated_data_annotations" / "responses"


with open(meta_dir / "id_to_name_gen.json", "r") as f:
    id_to_name_gen = json.load(f)

In [ ]:
# get all files from response_dir
response_files = list(response_dir.rglob("*.json"))
response_ids = sorted([f.stem for f in response_files])
print(f"{len(response_files)} response files out of {len(id_to_name_gen)} images")

In [ ]:
count_yes = {}

for response_file in response_files:
    id = response_file.stem
    if id in count_yes:
        continue
    else:
        count_yes[id] = 0
    with open(response_file, "r") as f:
        response_dict = json.load(f)
    question_list = response_dict["questions"]
    response_list = response_dict["responses"]
    for response in response_list:
        response = response.strip()[:5]  # only check the first 5 characters
        if ("YES" in response) or ("yes" in response) or ("Yes" in response):
            count_yes[id] += 1

In [ ]:
list_2y = []
list_1y = []
list_0y = []

for id in count_yes:
    if count_yes[id] == 2:
        list_2y.append(id)
    elif count_yes[id] == 1:
        list_1y.append(id)
    else:
        list_0y.append(id)

print(f"2 yes: {len(list_2y)}")
print(f"1 yes: {len(list_1y)}")
print(f"0 yes: {len(list_0y)}")

In [ ]:
no_lmm = set(list_0y) | set(list_1y)

In [ ]:
count_no = {}

for response_file in response_files:
    id = response_file.stem
    if id in count_no:
        continue
    else:
        count_no[id] = 0
    with open(response_file, "r") as f:
        response_dict = json.load(f)
    question_list = response_dict["questions"]
    response_list = response_dict["responses"]
    for response in response_list:
        response = response.strip()[:5]  # only check the first 5 characters
        if ("NO" in response) or ("no" in response) or ("No" in response):
            count_no[id] = count_no[id] + 1

In [ ]:
list_2n = []
list_1n = []
list_0n = []

for id in count_no:
    if count_no[id] == 2:
        list_2n.append(id)
    elif count_no[id] == 1:
        list_1n.append(id)
    else:
        list_0n.append(id)

print(f"2 no: {len(list_2n)}")
print(f"1 no: {len(list_1n)}")
print(f"0 no: {len(list_0n)}")

In [ ]:
assert no_lmm == set(list_2n) | set(list_1n)

In [ ]:
# save no_lmm in both meta_dir and repo_dir
with open(meta_dir / "no_lmm.json", "w") as f:
    json.dump(list(no_lmm), f)
with open(repo_dir / "no_lmm.json", "w") as f:
    json.dump(list(no_lmm), f)

In [ ]:
# copy to new folders
import shutil

new_image_dir = repo_dir / "generated_data_no_lmm"
for id in no_lmm:
    obj = id_to_name_gen[id + ".jpg"].split("/")[1].replace("_", " ")
    new_path = (new_image_dir / obj / id).with_suffix(".jpg")
    if new_path.exists():
        continue
    new_path.parent.mkdir(parents=True, exist_ok=True)
    old_path = (image_dir / obj / id).with_suffix(".jpg")
    shutil.copy(old_path, new_path)